In [1]:
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision import datasets, transforms
from collections import deque
from sklearn.metrics import root_mean_squared_error, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from UNeuralNetwork import LayerSequential, LinearLayer

In [2]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()

    output = model(x_batch.to('cpu'))  
    loss = loss_function(output, y_batch.to('cpu'))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()  

def train_epoch(train_generator, model, loss_function, optimizer, callback=None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to('cpu'), batch_of_y.to('cpu'), optimizer, loss_function)

        if callback is not None:
            callback(model, batch_loss)

        epoch_loss += batch_loss * len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss / total

def trainer(count_of_epoch,
            batch_size,
            dataset,
            model,
            loss_function,
            optimizer,
            lr=0.001,
            callback=None):
    optima = optimizer(model.parameters(), lr=lr)
    for it in range(count_of_epoch):
        batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
        epoch_loss = train_epoch(train_generator=batch_generator,
                                 model=model,
                                 loss_function=loss_function,
                                 optimizer=optima,
                                 callback=callback)
        print(f'Эпоха {it}: ошибка = {epoch_loss}') 

def tester(model, dataset, batch_size):
    batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size)
    y_predict = []
    y_real = []
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to('cpu')
        y_batch = y_batch.to('cpu')

        output = model(x_batch)
        y_predict.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
        y_real.extend(y_batch.cpu().numpy().tolist())
    print(classification_report(y_real, y_predict))    

In [3]:
class CNN(torch.nn.Module):
    @property
    def device(self):
        for param in self.parameters():
            return param.device

    def __init__(self):
        super(CNN, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('conv1', torch.nn.Conv2d(1, 1*1, kernel_size=5)) #Кол-во входных каналов(признаков, которые 
                                                                                #описывают объект)
                                                                                #Кол-во выходных каналов(признаков) 
                                                                                #Размер сверточного ядра(Оно сжимает изображение)
        self.layers.add_module('relu1', torch.nn.ReLU()) #Функция активации
        self.layers.add_module('pool1', torch.nn.MaxPool2d(kernel_size=2)) #Сжимает изображение ещё в 2 раза
        self.layers.add_module('conv2', torch.nn.Conv2d(1*1, 1*1, kernel_size=5))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('pool2', torch.nn.MaxPool2d(kernel_size=2))
        self.layers.add_module('flatten', torch.nn.Flatten()) #Вытягиваем тензор в один вектор
        self.layers.add_module('linear1', torch.nn.Linear(16*4*4, 120))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('linear2', torch.nn.Linear(120, 84))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('linear3', torch.nn.Linear(84, 10))

    def forward(self, input):
        return self.layers(input)

In [4]:
#Загрузка данных
MNIST_train = datasets.MNIST('DATA/mnist', train=True, download=True, transform=transforms.ToTensor())
MNIST_test = datasets.MNIST('DATA/mnist', train=False, download=True, transform=transforms.ToTensor())

In [5]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

model = CNN()
model.to('cpu')

CNN(
  (layers): Sequential(
    (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear1): Linear(in_features=256, out_features=120, bias=True)
    (relu3): ReLU()
    (linear2): Linear(in_features=120, out_features=84, bias=True)
    (relu4): ReLU()
    (linear3): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [6]:
#Обучение CNN
trainer(count_of_epoch=1,
        batch_size=64,
        dataset=MNIST_train,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.001,
        callback=None) #callback - позволяет отслеживать обучение модели(логирование). Используется TensorBoard

Эпоха 0: ошибка = 0.3092773442973693


In [7]:
tester(model=model,
       dataset=MNIST_test,
       batch_size=64)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.99      0.95      0.97      1032
           3       0.92      0.99      0.95      1010
           4       0.99      0.97      0.98       982
           5       0.97      0.96      0.96       892
           6       0.98      0.98      0.98       958
           7       0.97      0.96      0.96      1028
           8       0.99      0.93      0.96       974
           9       0.96      0.97      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



In [12]:
X = torch.tensor([[[1,0,3,5,3,7,6,3],
                   [9,4,3,8,1,3,2,1],
                   [0,3,5,2,0,5,6,4],
                   [3,1,5,0,3,4,3,3],
                   [2,2,0,1,0,1,2,0],
                   [0,2,1,0,2,3,4,3]],
                  [[7,6,3,2,0,1,2,1],
                   [1,2,7,4,2,3,2,7],
                   [3,4,2,0,2,0,1,2],
                   [0,1,2,0,0,4,3,0],
                   [0,1,9,1,9,2,3,2],
                   [9,1,2,3,9,0,2,1]]],
                 dtype=torch.float32)
y = torch.tensor([3])

In [11]:
class Conv2DLayer:
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 activation_function,
                 start_weights,
                 bias=True):
        self.activation_function = activation_function
        self.weights_ = self._start_weights_type[start_weights]((out_channels, in_channels, kernel_size, kernel_size))
        self.bias_ = self._start_weights_type[start_weights](out_channels,)

    @staticmethod
    def _makeConvolution(X_channel, kernel):
        k = kernel.shape[0]
        n = X_channel.shape[0] - k + 1
        m = X_channel.shape[1] - k + 1
        return torch.tensor([[torch.sum(X_channel[row:row+k, col:col+k]*kernel) for col in range(m)]
                             for row in range(n)]).reshape(1,n,m)

    def _makeNextImgChannel(self, X, kernels_for_channel, bias):
        result = torch.sum(torch.cat([self._makeConvolution(X_channel, kernel)
                                      for X_channel, kernel in zip(X, kernels_for_channel)]),
                           dim=-3) + bias
        n, m = result.shape
        return result.reshape(1,n,m)

    def _applyActivationFunction(self, X):
        return self._activation_function_type[self.activation_function](X)
    
    def layerForward(self, X):
        next_img = torch.cat([self._makeNextImgChannel(X, kernels_for_channels, bias)
                              for kernels_for_channels, bias in zip(self.weights_, self.bias_)])
        next_img = self._applyActivationFunction(next_img)
        return next_img

    @staticmethod
    def _linActivationF(X):
        return X
    
    @staticmethod
    def _reLUActivationF(X):
        return torch.where(X>=0, X, 0)

    @staticmethod
    def _tanhActivationF(X):
        return torch.tanh(X)
    
    @staticmethod
    def _zeroStartWeights(shape):
        return torch.zeros(shape)

    @staticmethod
    def _randomStartWeights(shape):
        return torch.randn(shape)
    
    _start_weights_type = {'zeros': _zeroStartWeights,
                           'random': _randomStartWeights}

    _activation_function_type = {'linear': _linActivationF,
                                 'ReLU': _reLUActivationF,
                                 'Tanh': _tanhActivationF}

class MaxPool2DLayer:
    def __init__(self, kernel_size=2):
        self.kernel_size = kernel_size

    def layerForward(self, X):
        return torch.cat([self._maxPooling(X_channel) for X_channel in X])
    
    def _maxPooling(self, X_channel):
        result = torch.tensor([[torch.max(X_channel[row:row+self.kernel_size, col:col+self.kernel_size]) 
                                for col in range(0, X_channel.shape[1], self.kernel_size)]
                               for row in range(0, X_channel.shape[0], self.kernel_size)])
        return result.reshape(1, result.shape[0], result.shape[1])

class FlattenLayer:
    def __init__(self):
        pass

    def layerForward(self, X):
        return X.reshape(-1,)

In [225]:
class UCNN(torch.nn.Module):
    def __init__(self):
        super(TargetModel, self).__init__()
        self.layers = torch.nn.Sequential()
        #self.layers.add_module('conv1', torch.nn.Conv2d(2, 3, kernel_size=3))
        #self.layers.add_module('relu1', torch.nn.ReLU())
        #self.layers.add_module('pool1', torch.nn.MaxPool2d(kernel_size=2))
        self.layers.add_module('flatten', torch.nn.Flatten(start_dim=0))
        self.layers.add_module('linear', torch.nn.Linear(96, 10))

    def forward(self, input):
        return self.layers(input)

    def getConvParams(self):
        return self.layers[0].weight, self.layers[0].bias

    def getLinearParams(self):
        return self.layers[-1].weight.T, self.layers[-1].bias

    def getModelParams(self):
        #print(f'-Сверточный слой\nВеса\n{self.layers[0].weight.data}\nСмещение\n{self.layers[0].bias.data}')
        print(f'-Линейный слой\nВеса\n{self.layers[-1].weight.data}\nСмещение\n{self.layers[-1].bias.data}')

In [235]:
target_model = TargetModel()
output = target_model.forward(X).reshape(1,-1)
target_model.getModelParams()
print(f'Результат {output}')

-Линейный слой
Веса
tensor([[ 2.7271e-02, -3.9024e-02, -7.6870e-02, -8.4852e-02, -8.7422e-02,
          4.9037e-02,  8.6713e-02,  1.1280e-02,  4.7331e-02,  3.4262e-02,
          5.2248e-02,  1.5519e-02,  1.0155e-02, -4.3080e-02, -4.6640e-02,
          3.5887e-02,  3.0978e-03, -8.9695e-02,  2.1046e-02, -8.6393e-02,
          3.4132e-02,  5.3289e-02,  6.0717e-02, -5.8975e-02,  1.8473e-02,
         -3.6538e-02, -2.7966e-02, -3.1913e-02, -4.2121e-02,  5.6044e-02,
          9.8480e-03,  5.2663e-02,  7.5648e-02,  3.3057e-02, -6.8974e-02,
         -7.3703e-02,  7.8590e-02, -7.5823e-02, -3.2988e-03,  1.1066e-02,
         -8.7328e-02, -6.9823e-02,  4.3512e-03, -2.0492e-02, -7.9301e-02,
          7.0097e-03,  5.3983e-02, -4.9685e-02,  8.6616e-02, -2.7284e-03,
         -8.2172e-02, -6.6727e-02, -8.4523e-02, -6.2193e-02, -1.1092e-02,
          6.5158e-02, -5.8784e-02,  2.4282e-02,  1.4000e-02, -1.0186e-01,
          1.6187e-02,  4.3798e-02,  1.9325e-02,  6.4360e-02,  3.9979e-02,
          9.5176e-

In [227]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(target_model.parameters(), lr=0.1)

loss = loss_function(output, y)
loss.backward()
optimizer.step()

target_model.getModelParams()

-Линейный слой
Веса
tensor([[-5.8971e-02,  6.8141e-02,  3.1033e-02, -1.2944e-02, -1.7911e-02,
          6.7453e-02,  2.4487e-02,  8.9585e-02, -8.8678e-02, -7.3682e-02,
         -1.0134e-01, -7.8189e-02,  5.1757e-02, -9.7536e-02, -2.1583e-02,
         -1.0133e-01, -4.5019e-02,  2.9161e-02,  3.7613e-02,  4.4074e-02,
          4.7085e-02, -2.2691e-02, -3.0753e-02,  7.3225e-02, -2.5000e-02,
          1.4456e-02, -9.1708e-02,  5.8287e-02,  6.3949e-02, -8.1443e-02,
         -5.1507e-02,  8.6361e-02, -4.9946e-02,  2.2743e-03,  5.6640e-02,
         -9.4879e-02, -9.9054e-02, -9.7921e-02, -7.1394e-02, -2.2157e-02,
          7.4123e-02, -6.7019e-02, -4.9240e-02,  2.4058e-02,  1.6054e-02,
         -7.6345e-04, -8.1809e-02,  8.2647e-02, -3.4222e-02,  2.5310e-02,
          6.0694e-02, -6.3084e-02, -5.5161e-02,  1.0152e-01, -1.2629e-02,
         -2.6366e-02, -8.2183e-02, -9.5567e-02, -4.6894e-02, -6.4551e-02,
          5.7070e-02, -3.8669e-02, -5.4934e-03, -8.0043e-02,  5.2746e-02,
          5.5442e-

In [15]:
conv = Conv2DLayer(2,5,3,'ReLU','random')
#conv.weights_, conv.bias_ = target_model.getConvParams()

#lin = LinearLayer(96, 10, 'linear', 'zeros')
#lin.weights_, lin.bias_ = target_model.getLinearParams()

Ulayers = LayerSequential()
Ulayers.add_module('Conv2D', conv)
Ulayers.add_module('maxPool2D', MaxPool2DLayer(2))
Ulayers.add_module('flatten', FlattenLayer())
#Ulayers.add_module('linear', lin)
Ulayers.forward(X)
output = Ulayers[-1].output_.reshape(1,-1)
print(f'Результат {output}')

AttributeError: 'FlattenLayer' object has no attribute 'output_'

In [65]:
class Conv2DLayer:
    def __init__(self, kernel_size, start_weights):
        self.kernel_size = kernel_size
        self.weights_ = self._start_weights_type[start_weights]((kernel_size, kernel_size))

    def __repr__(self):
        return f'Conv2DLayer(kernel_size=({self.kernel_size},{self.kernel_size}))'

    @staticmethod
    def _makeConvolution(channel, kernel):
        k = kernel.shape[0]
        n = channel.shape[0] - k + 1
        m = channel.shape[1] - k + 1
        return torch.tensor([[torch.sum(channel[row:row+k, col:col+k]*kernel) for col in range(m)]
                            for row in range(n)])

    def layerForward(self, X):
        return self._makeConvolution(X, self.weights_)

    @staticmethod
    def _zeroStartWeights(shape):
        return torch.zeros(shape)

    @staticmethod
    def _randomStartWeights(shape):
        return torch.randn(shape)
    
    _start_weights_type = {'zeros': _zeroStartWeights,
                           'random': _randomStartWeights}

In [68]:
class MaxPool2DLayer:
    def __init__(self, kernel_size=2):
        self.kernel_size = kernel_size

    def layerForward(self, X):
        return self._maxPooling(X)
    
    def _maxPooling(self, channel):
        result = torch.tensor([[torch.max(channel[row-self.kernel_size:row, col:col+self.kernel_size]) 
                                for col in range(0, channel.shape[1], self.kernel_size)]
                               for row in range(0, channel.shape[0], self.kernel_size)])
        return result

class FlattenLayer:
    def __init__(self):
        pass

    def layerForward(self, X):
        return X.reshape(-1,)

In [71]:
channel = torch.tensor([[1,0,3,5,4,5,6],
                        [9,4,3,8,8,7,3],
                        [0,3,5,2,4,3,2],
                        [3,1,5,0,1,7,4],
                        [2,2,0,1,0,2,2]],
                       dtype=torch.float32)

channel_torch = torch.tensor([[[1,0,3,5,4,5,6],
                               [9,4,3,8,8,7,3],
                               [0,3,5,2,4,3,2],
                               [3,1,5,0,1,7,4],
                               [2,2,0,1,0,2,2]]],
                             dtype=torch.float32)

In [75]:
seq = torch.nn.Sequential()
seq.add_module('conv2d', torch.nn.Conv2d(1,1,3,bias=False))
seq.add_module('max_pool', torch.nn.MaxPool2d(2))
print(f'Torch result\n{seq(channel_torch)}')

u_seq = LayerSequential()
u_seq.add_module('conv2d', Conv2DLayer(3, 'random'))
#u_seq.add_module('max_pool', MaxPool2DLayer(2))
u_seq[0].weights_ = seq[0].weight.data[0][0]
print(f'My result\n{u_seq.forward(channel)}')



Torch result
tensor([[[-0.7586, -0.3328]]], grad_fn=<MaxPool2DWithIndicesBackward0>)
My result
tensor([[-2.2781, -1.9299, -1.6958, -3.7991, -2.4279],
        [-0.7586, -2.2678, -0.3328, -2.0301, -4.2719],
        [-0.8114, -1.2358, -1.2547, -0.7473, -2.6943]])


In [41]:
seq[0]

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)

In [62]:
u_seq[0].weights_

tensor([[-0.3311,  0.0564, -0.3074],
        [ 0.2913,  0.0502, -0.1719],
        [ 0.0789, -0.1571,  0.2572]])

In [ ]:
torch.